# RLDT

## Step 1: Import the necessary libraries:

In [15]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [16]:
num_IOT_devices = 5

voltages_frequencies_IOT = [
    (1e6, 1.8),
    (2e6, 2.3),
    (4e6, 2.7),
    (8e6, 4.0),
    (16e6, 5.0),
    (32e6, 6.5),
]
num_MEC_devices = 10

voltages_frequencies_MEC = [
    (6 * 1e8, 0.8),
    (7.5 * 1e8, 0.825),
    (10 * 1e8, 1.0),
    (15 * 1e8, 1.2),
    (30 * 1e8, 2),
    (40 * 1e8, 3.1),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 100

#### *IOT*

In [17]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": 1,
        "voltages_frequencies": (10 * 1e8, 1.0),
        "ISL": np.random.randint(10, 21),
        "capacitance": np.random.uniform(2, 3) * 1e-9,
        "powerIdle": 900 * 1e-6 ,
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices

,number_of_cpu_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,
0,1,"(1000000000.0, 1.0)",10,2.997336e-09,0.0009,3.600000e+10,0.05,"[1, 2]",1
1,1,"(1000000000.0, 1.0)",10,2.979085e-09,0.0009,3.600000e+10,0.02,"[3, 2, 4, 1]",1
2,1,"(1000000000.0, 1.0)",15,2.351018e-09,0.0009,3.700000e+10,0.04,"[2, 4, 1, 3]",0
3,1,"(1000000000.0, 1.0)",12,2.348161e-09,0.0009,3.700000e+10,0.01,"[4, 1, 2, 3]",0
4,1,"(1000000000.0, 1.0)",16,2.916632e-09,0.0009,3.600000e+10,0.02,"[4, 2, 1, 3]",1


#### *MEC*

In [18]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": 1,
        "voltages_frequencies": (40 * 1e8, 3.1),
        "capacitance": np.random.uniform(1.5, 2) * 1e-9 ,
        "powerIdle": np.random.choice([9, 9, 10]) * 1e-5 ,
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices

,number_of_cpu_cores,voltages_frequencies,capacitance,powerIdle,errorRate,accetableTasks,handleSafeTask
id,,,,,,,
0,1,"(4000000000.0, 3.1)",1.893025e-09,0.00009,0.09,"[1, 2, 3, 4]",0
1,1,"(4000000000.0, 3.1)",1.908672e-09,0.00009,0.08,"[3, 1, 4, 2]",1
2,1,"(4000000000.0, 3.1)",1.699349e-09,0.00009,0.09,"[3, 4]",0
3,1,"(4000000000.0, 3.1)",1.909570e-09,0.00009,0.05,"[4, 2, 3, 1]",1
4,1,"(4000000000.0, 3.1)",1.747475e-09,0.00010,0.06,"[3, 4, 2]",0
5,1,"(4000000000.0, 3.1)",1.625219e-09,0.00010,0.08,"[4, 1, 3]",0
6,1,"(4000000000.0, 3.1)",1.628261e-09,0.00009,0.05,"[3, 1, 4]",0
7,1,"(4000000000.0, 3.1)",1.708824e-09,0.00009,0.07,"[1, 2, 4]",0
8,1,"(4000000000.0, 3.1)",1.590297e-09,0.00009,0.09,"[3, 1, 2]",0


#### *CLOUD*

In [19]:
cloud = (3.9e9, 2)

### Step 2.2: Application

#### *helper function : generate_random_dag*

In [20]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [21]:
tasks_data = []


for i in range(250000):
    # parents = list(random_dag.predecessors(node))
    task_info = {
        "id": i,
        "dependency": [],
        "mobility": np.random.randint(1, 10),
        "kind": np.random.choice(task_kinds),
        "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
        "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
        "dataEntrySize": (np.random.randint(10, 100) // 10)
        * (10 ** np.random.choice([3, 6])),
        "returnDataSize": (np.random.randint(10, 100) // 10)
        * (10 ** np.random.choice([3, 6])),
        "status": "READY",
    }
    tasks_data.append(task_info)
np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)

tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
14029,[],9,3,0,93559,6000,8000,READY
183255,[],7,4,0,197318,6000000,8000,READY
206798,[],9,4,1,112150,6000,7000000,READY
209464,[],2,4,0,16264,1000000,4000,READY
198797,[],5,4,0,862930,2000000,9000,READY
...,...,...,...,...,...,...,...,...
151795,[],8,1,0,481099,9000000,2000000,READY
159655,[],9,1,0,486257,6000000,4000,READY
65945,[],9,2,0,744316,1000000,8000000,READY


In [22]:
tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
14029,[],9,3,0,93559,6000,8000,READY
183255,[],7,4,0,197318,6000000,8000,READY
206798,[],9,4,1,112150,6000,7000000,READY
209464,[],2,4,0,16264,1000000,4000,READY
198797,[],5,4,0,862930,2000000,9000,READY
...,...,...,...,...,...,...,...,...
151795,[],8,1,0,481099,9000000,2000000,READY
159655,[],9,1,0,486257,6000000,4000,READY
65945,[],9,2,0,744316,1000000,8000000,READY


## Step 4 : DDT

### Step 4.1:  Initializing The tree

In [23]:
class DDTNode(nn.Module):
    def __init__(self, feature_size, num_classes, depth, max_depth):
        super(DDTNode, self).__init__()
        self.feature_size = feature_size
        self.num_classes = num_classes
        self.depth = depth
        self.max_depth = max_depth
        # learnable parameters
        self.weight = nn.Parameter(torch.ones(feature_size), requires_grad=True)
        self.bias = nn.Parameter(torch.ones(1), requires_grad=True)
        self.prob_distribution = nn.Parameter(
            torch.zeros(num_classes), requires_grad=True
        )
        # If not at leaf, create left and right child nodes
        if self.depth < self.max_depth:
            self.left_child = DDTNode(feature_size, num_classes, depth+1, max_depth)
            self.right_child = DDTNode(feature_size, num_classes, depth+1, max_depth)

    def forward(self, x):
        # Check if we are at a leaf node
        if self.depth == self.max_depth:
            # We are at a leaf, return the softmax probabilities for all instances in the batch
            probs = F.softmax(self.prob_distribution, dim=0)
            # print("probabilty distribution : ",probs)
            return probs.expand(x.size(0), -1)  
        else:
            # Process each item in the batch individually (not efficient!) TODO make it vectorized
            decisions = torch.sigmoid(torch.matmul(x, self.weight) + self.bias)
            batch_results = torch.zeros(x.size(0), self.num_classes)
            for i in range(x.size(0)):
                decision = decisions[i]
                if decision > 0.5:
                    batch_results[i] = self.right_child(x[i:i+1])
                else:
                    batch_results[i] = self.left_child(x[i:i+1])
            return batch_results

In [24]:
tasks_copy = tasks.copy()
tasks_copy = tasks_copy.drop(["dependency", "status"], axis=1)

In [25]:
class Environment:
    def __init__(self):
        self.feature_size = 6
        self.num_actions = 3  # Number of actions
        self.max_depth = 2  # Maximum depth of the decision tree
        self.agent = DDTNode(
            self.feature_size, self.num_actions, depth=0, max_depth=self.max_depth
        )
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.001)

    def get_reward(self, computationalLoad, action):
        if action == 0:
            executionTime = (
                computationalLoad / IoTdevices.iloc[0]["voltages_frequencies"][0]
            )
            powerConsumption = executionTime * (
                IoTdevices.iloc[0]["capacitance"]
                * (IoTdevices.iloc[0]["voltages_frequencies"][1] ** 2)
                * IoTdevices.iloc[0]["voltages_frequencies"][0]
            )
            return -1 * (executionTime + powerConsumption)
        elif action == 1:
            executionTime = (
                computationalLoad / MECDevices.iloc[0]["voltages_frequencies"][0]
            )
            powerConsumption = executionTime * (
                MECDevices.iloc[0]["capacitance"]
                * (MECDevices.iloc[0]["voltages_frequencies"][1] ** 2)
                * MECDevices.iloc[0]["voltages_frequencies"][0]
            )
            return -1 * (executionTime + powerConsumption)
        else:
            executionTime = computationalLoad / cloud[0]
            powerConsumption = executionTime * (1.28 * cloud[1] * cloud[0])
            return -1 * (executionTime + powerConsumption)

    def execute_action(self, tasks_copy):
        tasks_copy = tasks_copy.drop(tasks_copy.index[0], axis=0)
        return tasks_copy.iloc[0]

    def traverse(self, tree, depth):
        if hasattr(tree, "left_child"):
            self.traverse(tree.left_child, depth + 1)
        if hasattr(tree, "right_child"):
            self.traverse(tree.right_child, depth + 1)
        if depth == 3:
            print(tree.prob_distribution)

    def traverseAndGetP(self, tree, state):
        if tree.depth == tree.max_depth:
            return 1
        x = torch.tensor(state.values, dtype=torch.float32).unsqueeze(0)
        epsilon = 1e-7  # Small constant value
        decision = torch.sigmoid(torch.matmul(x, tree.weight) + tree.bias) + epsilon
        if decision > 0.5:
            return decision * self.traverseAndGetP(tree.right_child, state)
        else:
            return decision * self.traverseAndGetP(tree.left_child, state)

    def train(self, num_epoch, num_episodes):
        for _ in range(num_epoch):
            states = []
            probs = torch.tensor([], requires_grad=True)
            returns = torch.tensor([], requires_grad=True)
            for i in range(num_episodes):
                state = tasks_copy.iloc[0]

                action_probs = self.agent(
                    torch.tensor(state.values, dtype=torch.float32).unsqueeze(0)
                )
                next_state = self.execute_action(tasks_copy)
                states.append(state)
                rewards_temp = [
                    self.get_reward(state["computationalLoad"], 0),
                    self.get_reward(state["computationalLoad"], 1),
                    self.get_reward(state["computationalLoad"], 2),
                ]
                rewards = torch.tensor(rewards_temp)
                mean = rewards.mean()
                std = rewards.std()
                rewards_normalized = (rewards - mean) / std
                returns = torch.cat((returns, rewards_normalized), dim=0)

                # Assuming self.traverseAndGetP() returns a tensor of shape (batch_size, num_actions)
                p_values = self.traverseAndGetP(self.agent, state)
                p_values_expanded = p_values.unsqueeze(0).expand(
                    action_probs.size(0), -1
                )  # Expanding p_values to match the batch size
                probs = torch.cat((probs, p_values_expanded * action_probs), dim=0)
                state = next_state
            returns = returns.reshape(1000, -1)
            loss = -(probs * returns).sum()
            # print(loss,self.traverse(self.agent,1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()  # Update the model parameters
            for name, param in self.agent.named_parameters():
                print(name, param.grad)
            print("----------------------------")
            # return


env = Environment()
tree = env.agent
env.train(25, 1000)

weight tensor([0., 0., 0., 0., 0., 0.])
bias tensor([0.])
prob_distribution None
left_child.weight None
left_child.bias None
left_child.prob_distribution None
left_child.left_child.weight None
left_child.left_child.bias None
left_child.left_child.prob_distribution None
left_child.right_child.weight None
left_child.right_child.bias None
left_child.right_child.prob_distribution None
right_child.weight tensor([0., 0., 0., 0., 0., 0.])
right_child.bias tensor([0.])
right_child.prob_distribution None
right_child.left_child.weight None
right_child.left_child.bias None
right_child.left_child.prob_distribution None
right_child.right_child.weight None
right_child.right_child.bias None
right_child.right_child.prob_distribution tensor([-192.4487, -192.4487,  384.8973])
----------------------------
weight tensor([0., 0., 0., 0., 0., 0.])
bias tensor([-0.])
prob_distribution None
left_child.weight None
left_child.bias None
left_child.prob_distribution None
left_child.left_child.weight None
left_chi

KeyboardInterrupt: 